# Projeto de ciência de dados - Airbnb Rio 
## Ferramenta de previsão de preço de imóvel para pessoas comuns 

### Esse projeto foi realizado pelo curso Python Impressionador da Hashtag Treinamentos. 
### Curso de Python e site da Hashtag: https://www.hashtagtreinamentos.com/curso-python

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### Inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

In [ ]:
# Imports
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
import joblib
import atexit
from time import time, strftime, localtime
from datetime import timedelta


## As funções abaixo são usadas apenas para obter o tempo de execução de cada um dos modelos de machine learning

In [ ]:
def secondsToStr(elapsed = None):
    
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds = elapsed))

def log(s, elapsed = None):
    
    line = "=" * 40
    print(line)
    print(secondsToStr(), '-', s)
    
    if elapsed:
        print("Tempo de execução:", elapsed)
    
    print(line)
    print()

def endlog():

    end = time()
    elapsed = end-start
    log("Fim do modelo!", secondsToStr(elapsed))


### Importando as bases de dados

Foi criado um dicionário com os 3 primeiros caracteres com os nomes dos meses e o número correspondente daquele mês.

Para cada arquivo da base de dados será importado o arquivo e criada uma coluna na base de dados com o mês e o ano de cada informação

In [ ]:
meses = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}

caminho_bases = pathlib.Path('dataset')

base_airbnb = pd.DataFrame()

for arquivo in caminho_bases.iterdir():
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    
    ano = arquivo.name[-8:]
    ano = int(ano.replace('.csv', ''))
    
    df = pd.read_csv(caminho_bases / arquivo.name, low_memory = False)
    df['ano'] = ano
    df['mes'] = mes
    base_airbnb = base_airbnb.append(df)

display(base_airbnb)

### Tratamentos

- Como existem muitas colunas, o modelo pode acabar ficando muito lento.

- Além disso, uma análise rápida permite ver que várias colunas não são necessárias para o nosso modelo de previsão, por isso, serão excluídas algumas colunas da base.

- Tipos de colunas que serão excluídas:
    
    1. IDs, Links e informações não relevantes para o modelo.
    
    2. Colunas repetidas ou extremamente parecidas com outra (que dão a mesma informação para o modelo. Ex: Data x Ano/Mês.
    
    3. Colunas preenchidas com texto livre.
    
    4. Colunas em que todos ou quase todos os valores são iguais.
    
- Para isso, será criado um arquivo em excel com os 1.000 primeiros registros e será feita uma análise qualitativa, olhando as colunas e identificando quais são desnecessárias.

In [ ]:
print(list(base_airbnb.columns))
base_airbnb.head(1000).to_csv('primeiros_registros.csv', sep = ';')

### Depois da análise qualitativa das colunas, levando em conta os critérios explicados acima, as seguintes colunas serão as usadas no modelo:


In [ ]:
colunas = ['host_response_time', 'host_response_rate', 'host_is_superhost', 'host_listings_count', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy', 'ano', 'mes']

base_airbnb = base_airbnb.loc[:, colunas]

print(list(base_airbnb.columns))

display(base_airbnb)

### Tratamento de valores vazios

- Visualizando os dados, percebe-se que existe uma grande disparidade em dados faltantes. As colunas com mais de 300.000 valores NaN foram excluídas da análise.

- Para as outras colunas, como existem muitos dados (mais de 900.000 linhas) serão excluídas as linhas que contém dados NaN.

In [ ]:
for coluna in base_airbnb:
    if base_airbnb[coluna].isnull().sum() > 300000:
        base_airbnb = base_airbnb.drop(coluna, axis = 1)
print(base_airbnb.isnull().sum())

- Agora serão excluídas as linhas vazias

In [ ]:
base_airbnb = base_airbnb.dropna()

print(base_airbnb.shape)
print(base_airbnb.isnull().sum())

### Verificação de tipos de dados em cada coluna

- Para garantir que todas as colunas estejam sendo lidas de acordo

In [ ]:
print(base_airbnb.dtypes)
print('-' * 60)
print(base_airbnb.iloc[0])

- Como preço e extra people estão sendo reconhecidos como objeto (ao invés de ser um float) mudarei o tipo de variável da coluna.

In [ ]:
# Price
base_airbnb['price'] = base_airbnb['price'].str.replace('$', '')
base_airbnb['price'] = base_airbnb['price'].str.replace(',', '')
base_airbnb['price'] = base_airbnb['price'].astype(np.float32, copy = False)

# Extra people
base_airbnb['extra_people'] = base_airbnb['extra_people'].str.replace('$', '')
base_airbnb['extra_people'] = base_airbnb['extra_people'].str.replace(',', '')
base_airbnb['extra_people'] = base_airbnb['extra_people'].astype(np.float32, copy = False)

# Verificando os tipos
print(base_airbnb.dtypes)

### Análise exploratória e tratamento de outliers

- Cada feature será analisada para:

    1. Ver a correlação entre as features e decidir se serão mantidas as features que temos.
    
    2. Excluir outliers (será usado como regra, valores abaixo de Q1 - 1.5x Amplitude e valores acima de Q3 + 1.5x Amplitude). Amplitude = Q3 - Q1.
    
    3. Confirmar se todas as features fazem realmente sentido para o modelo ou se alguma delas não vai ajudar e deve ser excluída.

In [ ]:
plt.figure(figsize = (15, 10))
sns.heatmap(base_airbnb.corr(), annot = True, cmap = 'Greens')

### Definição das funções para analisar os outliers

Vamos definir algumas funções para ajudar na análise de outliers das colunas

In [ ]:
def limites(coluna):

    q1 = coluna.quantile(0.25)
    q3 = coluna.quantile(0.75)
    amplitude = q3 - q1
    
    return q1 - 1.5 * amplitude, q3 + 1.5 * amplitude

def excluir_outliers(df, nome_coluna):
    
    qtde_linhas = df.shape[0]
    lim_inf, lim_sup = limites(df[nome_coluna])
    df = df.loc[(df[nome_coluna] >= lim_inf) & (df[nome_coluna] <= lim_sup), :]
    linhas_removidas = qtde_linhas - df.shape[0]
    
    return df,  linhas_removidas

In [61]:
# As funções abaixo plotam gráficos de diferentes tipos

def diagrama_caixa(coluna):
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15, 5)
    sns.boxplot(x = coluna, ax = ax1)
    ax2.set_xlim(limites(coluna))
    sns.boxplot(x = coluna, ax = ax2)
    
def histograma(coluna):
    
    plt.figure(figsize = (15, 5))
    sns.distplot(coluna, hist = True)

def grafico_barra(coluna):  
    
    plt.figure(figsize = (15, 5))
    ax = sns.barplot(x = coluna.value_counts().index, y = coluna.value_counts())
    ax.set_xlim(limites(coluna))

### Plotando gráfico para a análise de outliers de cada coluna:

- ### Coluna 'price'

In [ ]:
diagrama_caixa(base_airbnb['price'])
histograma(base_airbnb['price'])

Como o modelo é para imóveis comuns, acredito que os valores acima do limite superior serão apenas de imóveis de altíssimo luxo, que não é o objetivo principal. Por isso, serão excluídos esses outliers.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'price')
print('{} linhas removidas'.format(linhas_removidas))

In [ ]:
histograma(base_airbnb['price'])
print(base_airbnb.shape)

- ### Coluna 'extra_people'

In [ ]:
diagrama_caixa(base_airbnb['extra_people'])
histograma(base_airbnb['extra_people'])

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'extra_people')
print('{} linhas removidas'.format(linhas_removidas))

In [ ]:
histograma(base_airbnb['extra_people'])
print(base_airbnb.shape)

- ### Coluna 'host_listings_count'

In [ ]:
diagrama_caixa(base_airbnb['host_listings_count'])
grafico_barra(base_airbnb['host_listings_count'])

Serão excluídos os outliers, pois para o objetivo do projeto hosts com mais de 6 imóveis no airbnb não são o público alvo do objetivo do projeto (imagino que sejam imobiliários ou profissionais que gerenciam imóveis no airbnb)

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'host_listings_count')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'accommodates'

In [ ]:
diagrama_caixa(base_airbnb['accommodates'])
grafico_barra(base_airbnb['accommodates'])

- Pelo mesmo motivo do "host_listings_count" serão excluídos os outliers dessa coluna pois apartamentos que acomodam mais de 9 pessoas provavelmente não são imóveis comuns.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'accommodates')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'bathrooms' 

In [ ]:
diagrama_caixa(base_airbnb['bathrooms'])
plt.figure(figsize = (15, 5))
sns.barplot(x = base_airbnb['bathrooms'].value_counts().index, y = base_airbnb['bathrooms'].value_counts())

- Pelo mesmo motivo dos anteriores, serão excluídos os outliers nos banheiros.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'bathrooms')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'bedrooms'

In [ ]:
diagrama_caixa(base_airbnb['bedrooms'])
grafico_barra(base_airbnb['bedrooms'])

- Pelo mesmo motivo dos anteriores, serão excluídos os outliers em quantidade de quartos.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'bedrooms')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'beds'

In [ ]:
diagrama_caixa(base_airbnb['beds'])
grafico_barra(base_airbnb['beds'])

- Pelo mesmo motivo dos anteriores, serão excluídos os outliers em quantidade de camas.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'beds')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'guests_included'

In [ ]:
print(limites(base_airbnb['guests_included']))
plt.figure(figsize = (15, 5))
sns.barplot(x = base_airbnb['guests_included'].value_counts().index, y = base_airbnb['guests_included'].value_counts())

Essa feature será removida da análise. Ao que tudo indica, os usuários do airbnb usam muito o valor padrão do airbnb como 1 guest included. Isso pode levar o nosso modelo a considerar uma feature que na verdade não é essencial para a definição do preço, por isso, me parece melhor excluir a coluna da análise.

In [ ]:
base_airbnb = base_airbnb.drop('guests_included', axis = 1)
base_airbnb.shape

- ### Coluna 'minimum_nights'

In [ ]:
diagrama_caixa(base_airbnb['minimum_nights'])
grafico_barra(base_airbnb['minimum_nights'])

- Estamos querendo um modelo que ajude a precificar apartamentos comuns como uma pessoa comum gostaria de disponibilizar. No caso, apartamentos com mais de 8 noites como o "mínimo de noites" podem ser apartamentos de temporada ou ainda apartamentos para morar, em que o host exige pelo menos 1 mês no apartamento.

- Por isso, serão excluídos os outliers dessa coluna.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'minimum_nights')
print('{} linhas removidas'.format(linhas_removidas))

- ### Coluna 'maximum_nights'

In [ ]:
diagrama_caixa(base_airbnb['maximum_nights'])
grafico_barra(base_airbnb['maximum_nights'])

- Ao que tudo indica quase todos os hosts não preenchem esse campo de maximum nights, portanto, não parece ser um fator relevante.

- Essa coluna será excluída da análise.

In [ ]:
base_airbnb = base_airbnb.drop('maximum_nights', axis = 1)
base_airbnb.shape

- ### Coluna 'number_of_reviews'            

In [ ]:
diagrama_caixa(base_airbnb['number_of_reviews'])
grafico_barra(base_airbnb['number_of_reviews'])

Aqui a coluna será removida do modelo, levando em consideração que o objetivo do projeto é para casas e pessoas comuns, é altamente provável que as pessoas que usariam esse modelo estariam anunciando sua casa/apartamento pela primeira vez e portanto não teriam reviews ainda.

In [ ]:
base_airbnb = base_airbnb.drop('number_of_reviews', axis = 1)
base_airbnb.shape

### Tratamento das colunas de valores de texto

- ### Coluna 'property_type' 

In [ ]:
print(base_airbnb['property_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('property_type', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

- Aqui serão agrupados valores muito pequenos.

- Todos os tipos de propriedade que têm menos de 2.000 propriedades na base de dados, serão agrupados em um grupo chamado "outros".

In [ ]:
tabela_tipos_casa = base_airbnb['property_type'].value_counts()
colunas_agrupar = []

for tipo in tabela_tipos_casa.index:
    if tabela_tipos_casa[tipo] < 2000:
        colunas_agrupar.append(tipo)

print(colunas_agrupar)

for tipo in colunas_agrupar:
    base_airbnb.loc[base_airbnb['property_type'] == tipo, 'property_type'] = 'Outros'

print(base_airbnb['property_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('property_type', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

- ### Coluna 'room_type' 

In [ ]:
print(base_airbnb['room_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('room_type', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

Nessa coluna, não farei nada, ela já parece estar bem distribuído

- ### Coluna 'bed_type' 

In [ ]:
print(base_airbnb['bed_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('bed_type', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

# Agrupando as categorias de tipo de cama ou 'bed_type'
tabela_bed = base_airbnb['bed_type'].value_counts()
colunas_agrupar = []

for tipo in tabela_bed.index:
    if tabela_bed[tipo] < 10000:
        colunas_agrupar.append(tipo)

print(colunas_agrupar)

for tipo in colunas_agrupar:
    base_airbnb.loc[base_airbnb['bed_type'] == tipo, 'bed_type'] = 'Outros'

print(base_airbnb['bed_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('bed_type', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

- Aqui serão agrupados valores muito pequenos.

- Como existe um valor muito acima dos outros, vou criar apenas 2 grupos de camas: "Real Bed" e "outros"

- Essa ação simplificará o modelo.

- ### Coluna 'cancellation_policy' 

In [ ]:
print(base_airbnb['cancellation_policy'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('cancellation_policy', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

# Agrupando as categorias de 'cancellation_pollicy'
tabela_cancellation = base_airbnb['cancellation_policy'].value_counts()
colunas_agrupar = []

for tipo in tabela_cancellation.index:
    if tabela_cancellation[tipo] < 10000:
        colunas_agrupar.append(tipo)

print(colunas_agrupar)

for tipo in colunas_agrupar:
    base_airbnb.loc[base_airbnb['cancellation_policy'] == tipo, 'cancellation_policy'] = 'strict'

print(base_airbnb['cancellation_policy'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('cancellation_policy', data = base_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

- ### Coluna 'amenities' 

Como existe uma infinidade de amenities e muitas vezes as mesmas amenities são escritas de formas diferentes, esse modelo levará em consideração apenas a quantidade de amenities e não as amenities em si.

In [ ]:
print(base_airbnb['amenities'].iloc[1].split(','))
print(len(base_airbnb['amenities'].iloc[1].split(',')))

base_airbnb['n_amenities'] = base_airbnb['amenities'].str.split(',').apply(len)

In [ ]:
base_airbnb = base_airbnb.drop('amenities', axis = 1)
base_airbnb.shape

In [ ]:
diagrama_caixa(base_airbnb['n_amenities'])
grafico_barra(base_airbnb['n_amenities'])

Agora essa coluna é uma coluna de valores numéricos (a quantidade de amenities), serão excluídos todos os outliers assim como nas outras colunas de valores numéricos tratadas anteriormente nesse modelo.

In [ ]:
base_airbnb, linhas_removidas = excluir_outliers(base_airbnb, 'n_amenities')
print('{} linhas removidas'.format(linhas_removidas))

### Visualização de mapa das propriedades

Será criado um mapa que exibe um pedaço aleatório da base de dados (50.000 propriedades) para ver como as propriedades estão distribuídas pela cidade e também identificar os locais de maior preço.

In [ ]:
amostra = base_airbnb.sample(n = 50000)
centro_mapa = {'lat':amostra.latitude.mean(), 'lon':amostra.longitude.mean()}
mapa = px.density_mapbox(amostra, lat = 'latitude', lon = 'longitude', z = 'price', radius = 2.5,
                        center = centro_mapa, zoom = 10,
                        mapbox_style = 'stamen-terrain')

mapa.show()

### Encoding

Aqui serão ajustadas as features para facilitar o trabalho do modelo futuro (features de categoria, true e false, etc.)

- Features de Valores True ou False, serão substituidos True por 1 e False por 0.
- Features de Categoria (features em que os valores da coluna são textos) será utilizado o método de encoding de variáveis dummies

In [ ]:
colunas_tf = ['host_is_superhost', 'instant_bookable', 'is_business_travel_ready']
base_airbnb_cod = base_airbnb.copy()

for coluna in colunas_tf:
    base_airbnb_cod.loc[base_airbnb_cod[coluna] == 't', coluna] = 1
    base_airbnb_cod.loc[base_airbnb_cod[coluna] == 'f', coluna] = 0

In [ ]:
colunas_categorias = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']
base_airbnb_cod = pd.get_dummies(data = base_airbnb_cod, columns = colunas_categorias)

display(base_airbnb_cod.head())

### Modelo de Previsão

- Métricas de Avaliação

Será usado aqui o 'R²' que dira o quão bem o modelo consegue explicar os preços. Isso seria um ótimo parâmetro para ver o quão bom é o modelo. <br>

-> Quanto mais próximo de 100%, melhor

Também será calculado o 'Erro Quadrático Médio', que mostrará o quanto o modelo está errando. <br>

-> Quanto menor for o erro, melhor

In [62]:
def avaliar_modelo(nome_modelo, y_teste, previsao):
    r2 = r2_score(y_teste, previsao)
    RSME = np.sqrt(mean_squared_error(y_teste, previsao))
    
    return f'Modelo {nome_modelo}:\nR²:{r2:.2%}\nRSME:{RSME:.2f}'

- Os modelos que serão testados são:
    
    1. RandomForest
    
    2. LinearRegression
    
    3. Extra Tree

In [ ]:
modelo_rf = RandomForestRegressor()
modelo_lr = LinearRegression()
modelo_et = ExtraTreesRegressor()

modelos = {'RandomForest': modelo_rf,
          'LinearRegression': modelo_lr,
          'ExtraTrees': modelo_et,
          }

y = base_airbnb_cod['price']
X = base_airbnb_cod.drop('price', axis = 1)

- Serão separados os dados em dados de treino e dados de teste, e após isso, o modelo será treinado e testado usando esses dados.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

for nome_modelo, modelo in modelos.items():
    
    # Treinar
    modelo.fit(X_train, y_train)
    
    # Testar
    start = time()
    log('Começo da execução:', nome_modelo)

    previsao = modelo.predict(X_test)
    print(avaliar_modelo(nome_modelo, y_test, previsao))

    endlog()

### Análise do melhor modelo

In [ ]:
for nome_modelo, modelo in modelos.items():
    
    # Testar
    start = time()
    log('Começo da execução:', nome_modelo)
    
    previsao = modelo.predict(X_test)
    
    print(avaliar_modelo(nome_modelo, y_test, previsao))
    endlog()

- O melhor modelo foi o: Extra Trees Regressor

    Esse foi o modelo com maior valor de R² e ao mesmo tempo o menor valor de RSME. Como não houve uma grande diferença de velocidade de treino e de previsão desse modelo com o modelo de RandomForest (que teve resultados próximos de R² e RSME), será escolhido o Modelo Extra Trees.
    
    O modelo de regressão linear não obteve um resultado satisfatório, com valores de R² e RSME muito piores do que os outros 2 modelos.
    
- Resultados das métricas de avaliação no modelo vencedor:<br>
Modelo ExtraTrees:<br>

R²: 97.49%<br>

RSME: 41.99

### Ajustes e melhorias no modelo escolhido (melhor modelo)

In [ ]:
importancia_features = pd.DataFrame(modelo_et.feature_importances_, X_train.columns)
importancia_features = importancia_features.sort_values(by = 0, ascending = False)

display(importancia_features)

plt.figure(figsize = (15, 5))
ax = sns.barplot(x = importancia_features.index, y = importancia_features[0])
ax.tick_params(axis = 'x', rotation = 90)

### Ajustes finais no modelo

- A coluna 'is_business_travel_ready' não parece ter muito impacto no modelo. Por isso, para chegar em um modelo mais simples, essa feature será excluída e o modelo testado novamente sem ela..

In [ ]:
base_airbnb_cod = base_airbnb_cod.drop('is_business_travel_ready', axis = 1)

y = base_airbnb_cod['price']
X = base_airbnb_cod.drop('price', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

modelo_et.fit(X_train, y_train)
previsao = modelo_et.predict(X_test)

print(avaliar_modelo('ExtraTrees', y_test, previsao))

In [ ]:
base_teste = base_airbnb_cod.copy()

for coluna in base_teste:
    if 'bed_type' in coluna:    
        base_teste = base_teste.drop(coluna, axis = 1)

print(base_teste.columns)

y = base_teste['price']
X = base_teste.drop('price', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

modelo_et.fit(X_train, y_train)
previsao = modelo_et.predict(X_test)

print(avaliar_modelo('ExtraTrees', y_test, previsao))

In [ ]:
print(previsao)

# Deploy do projeto

In [ ]:
X['price'] = y
X.to_csv('dados.csv')

In [ ]:
joblib.dump(modelo_et, 'modelo.joblib')